<div style="width:100%; text-align:center; margin: 40px 0;">
    <h1 style="color: #003366; font-size: 40px; font-weight:bold; text-shadow: 2px 2px 5px grey;">
        Énergie Électrique en France
    </h1>
    <hr style="width: 50%; height: 3px; background-color: #003366; border: none;">
    <p style="font-style:italic; font-size:18px; color: #555;">
        Une exploration approfondie de l'énergie électrique en France
    </p>
</div>


## Données:

Dans ce projet nous allons travailler sur des données produites par RTE France. Nous allons récolter ces données grâce à une API (OpenData Reseaux Energie https://opendata.reseaux-energies.fr/). Nous allons travailler sur des données régionales récoltées au niveau régional et sur des données nationales. Nous aurons aussi des API qui contiennent des données en temps réel, ce qui nous permettra d'avoir des données mise à jour régulierement.  

Voici les 4 liens API que nous utiliserons durant ce projet:
- Données Nationales en temps réel (du 1er Juin 2022 à aujourd'hui): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-tr`
- Données Nationales fixes (jusqu'au 31 Mai 2022): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-cons-def`
- Données Régionales en temps réel (du 1er Juin 2022 à aujourd'hui): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr`
- Données Régionales fixes (jusqu'au 31 Mai 2022): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-cons-def`

Grâce au module `request` de python nous allons pouvoir faire des requêtes de type SQL à ces API dans le but d'obtenir les données dont nous avons besoin au format JSON.

Nous utiliserons aussi un fichier GeoJson récupéré sur le site `https://france-geojson.gregoiredavid.fr/`. Ce fichier possède les données géographiques de la France, par région ou par département. Les données de RTE France étant uniquement nationales et régionales, nous utiliserons uniquement le GeoJson des régions de la France.

### Variables que nous allons analyser

- **Date** : *Date à laquelle les données ont ete enregistrees*
- **Heure** : *Date à laquelle les données ont ete enregistrees*
- **Région** (pour les données régionales) : *La région correspondant aux données*
- **Consommation** (MW) : *La consommation électrique réelle*
- **Prévision**  Conso J et J-1 (MW) :  *La consommation électrique prédite par RTE France au jours J et J-1*
- **Production par filières** (MW) : *La production d'energie electrique pqr filiere (nucleaire, hydrolique, eolien, solaire...)*
- **Imports exports  avec plusieurs pays** (MW) : *Echanges d'energie avec les differents pays frontaliers (Suisse, Allemagne, Belgique, Angleterre, Espagne)*
- **Emission de CO2** (g/kWh) : *La valeur d'emission de CO2*

## Type d'analyse prévue

Nous allons effectuer plusieurs visualisations sur nos données afin d'en tirer des observations. Pour cela nous allons principalement utiliser le module `plotly express` qui permet de faire des graphiques interactifs. Nous allons également améliorer l'interactivitée de notre graphique en rajoutant des modules IPython permetant à l'utilisateur de pouvoir trier et filtrer les données.  
Nous effecturons également des analyses statistiques afin de repondre à certains questionnements sur l'électricitée en France.

Pour cela nous concentrerons notre analyse sur 3 axes principaux:
- La consommation électrique en France
- La production d'énergie électrique aux niveaux national et régional
- Les échanges commerciaux avec les pays frontaliers

Nous allons d'abord devoir récupérer, nettoyer et formater les données lors des requêtes à l'API d'OpenData. Nous utiliserons ensuite Python et les bibliothèques Pandas et Plotly pour manipuler ces données et créer des visualisations.

## Résultats attendus

A la fin de cette analyse, nous devrions être capable de répondre à plusieurs questions tel que:
- Quels sont les modes de production les plus importants en France ? Par régions ?
- Comment a évolué la part du renouvelable dans le mix énergétique français durant la dernière décennie ?
- Quel est la balance commerciale de la France pour l’énergie ?
- Comment évolue la production et la consommation de l’électricité  en France  durant 1 journée ? 1 mois ? 1 année ?

Voici le type de graphique que l'on pourrait avoir à la fin de notre analyse:  

Voici un graphique représentant la consommation d'énergie électrique en France sur une seule journée.

![Consommation électrique](IMG/conso.png)

Ensuite nous pourrions avoir un graphique qui montre les échanges d'électricité avec les pays frontaliers.  
  
![Echanges commerciaux](IMG/echanges_comm.png)

Ensuite voici un graphique qui montre la proction d'energie électrique par la filière solaire sur une journée. Nous pouvons voir que le pic de production se situe vers 12h ce qui correspond au moment ou il y a le plus de soleil dans la journée.  
  
![Production d'energie solaire](IMG/production_solaire.png)  

Et pour finir voici un graphique representant la part de production d'énergie électrique pour chaque filiere:  
  
![Production d'energie par filiere](IMG/pie_chart_production.png)  


In [2]:
# Data

URL = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/"

# Imports
import requests
import json

import pandas as pd

import plotly.express as px


In [3]:
def fetch_data_by_date(data:str, start:int, rows:int, date:str):
    """Fetch data from a dataset by date and offset
    
    Parameters
    ----------
    data : str
        Name of the dataset.
    start : int
        Offset.
    rows : int
        Number of rows.
    date : str
        Date of the data.

    Returns
    -------
    dict
        Dictionary containing the data. 
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "offset" : start,
        "rows": rows,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results', {})
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return 
    
def get_length_per_date(data:str, date:str):
    """Get the number of rows for a given date

    Parameters
    ----------
    data : str
        Name of the dataset.
    date : str
        Date of the data.

    Returns
    -------
    int
        Number of rows.
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('total_count')
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return
      

In [6]:
def insert_data_into_json(file_name: str, new_data: list):
    """Insère de nouvelles données dans un fichier JSON existant.

    Parameters
    ----------
    file_name : str
        Le nom du fichier JSON.
    new_data : list
        La liste des nouvelles données à insérer.
    """
    try:
        with open(file_name, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    existing_data.extend(new_data)

    with open(file_name, 'w') as file:
        json.dump(existing_data, file)

for i in range(1, 31):
    date = f"2023-11-{i:02}"
    length = get_length_per_date('eco2mix-national-tr', date)
    print(f"{date} : {length}")
    new_data = fetch_data_by_date('eco2mix-national-tr', 0, length, date)
    insert_data_into_json('combined_data.json', new_data)

2023-11-01 : 96
2023-11-02 : 96
2023-11-03 : 96
2023-11-04 : 96
2023-11-05 : 96
2023-11-06 : 96
2023-11-07 : 96
2023-11-08 : 96
2023-11-09 : 96
2023-11-10 : 96
2023-11-11 : 96
2023-11-12 : 96
2023-11-13 : 96
2023-11-14 : 96
2023-11-15 : 96
2023-11-16 : 96
2023-11-17 : 96
2023-11-18 : 96
2023-11-19 : 96
2023-11-20 : 96
2023-11-21 : 96
2023-11-22 : 96
2023-11-23 : 96
2023-11-24 : 96
2023-11-25 : 96
2023-11-26 : 96
2023-11-27 : 96
2023-11-28 : 96
2023-11-29 : 96


In [5]:
def transform_json_to_df(file_name:str):
    """Transform a Json File to a pandas DataFrame"""
    with open(file_name) as f:
        data = json.load(f)
    return pd.DataFrame(data)
    
df = transform_json_to_df('combined_data.json')
df

,perimetre,nature,date,heure,date_heure,consommation,prevision_j1,prevision_j,fioul,charbon,...,gaz_ccg,gaz_autres,hydraulique_fil_eau_eclusee,hydraulique_lacs,hydraulique_step_turbinage,bioenergies_dechets,bioenergies_biomasse,bioenergies_biogaz,stockage_batterie,destockage_batterie
0,France,Données temps réel,2023-11-01,02:15,2023-11-01T01:15:00+00:00,39963,40200,40400,227,6,...,222,0,4929,1537,38,80,353,245,-3,0
1,France,Données temps réel,2023-11-01,03:30,2023-11-01T02:30:00+00:00,37128,36700,37300,229,6,...,229,0,5006,1578,24,87,353,245,-12,3
2,France,Données temps réel,2023-11-01,06:30,2023-11-01T05:30:00+00:00,37897,38000,38200,227,6,...,137,0,4748,1361,126,85,356,245,-2,2
3,France,Données temps réel,2023-11-01,07:15,2023-11-01T06:15:00+00:00,38876,38800,38950,225,6,...,192,0,4781,1543,375,82,353,245,-9,2
4,France,Données temps réel,2023-11-01,08:00,2023-11-01T07:00:00+00:00,39659,39300,39700,222,6,...,217,0,4809,1732,323,82,355,245,-2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,France,Données temps réel,2023-11-29,18:45,2023-11-29T17:45:00+00:00,70523,71550,70950,679,321,...,5615,0,6348,4501,3678,74,506,246,-7,2
5564,France,Données temps réel,2023-11-29,19:00,2023-11-29T18:00:00+00:00,70970,72200,71700,790,389,...,5637,0,6341,4455,3706,76,507,246,-8,2
5565,France,Données temps réel,2023-11-29,20:15,2023-11-29T19:15:00+00:00,67068,67950,67450,788,550,...,5763,0,5998,4125,2688,74,513,246,-1,0
5566,France,Données temps réel,2023-11-29,21:00,2023-11-29T20:00:00+00:00,64352,64700,64200,445,514,...,5557,0,5903,3807,1636,75,516,246,-2,0


In [21]:
from ipywidgets import interact_manual
import plotly.express as px

@interact_manual(day=(1,30))
def line_chart_consumption_november(day = 1):
    """Display a line chart of the consumption in France for a given day in November 2023.

    Parameters
    ----------
    day : int
        Day of the month.

    Returns
    -------
    plotly.graph_objects.Figure
        Line chart of the consumption in France for a given day in November 2023.

    """
    df = transform_json_to_df('combined_data.json')
    df = df[df['date'] == f'2023-11-{day:02}']
    fig = px.line(df, x="heure", y="consommation", title=f'Consommation électrique en France le {day:02} novembre 2023')
    fig.show()

interactive(children=(IntSlider(value=1, description='day', max=30, min=1), Button(description='Run Interact',…